## Sequence Classification

## Task 1.1: Document-level Sentiment Classification

Build a Bidirectional Recurrent Neural Network (RNN) model for multi-class sentiment classification. Compare the performance with a Unidirectional RNN model. Your model (each) shall
include:

- RNN network that learns sentence representation from input sequences.
- Fully connected network that predicts sentiment label, given the learnt state representation.


Train the model by using data iterator and batch generator. Evaluate the trained model on
the provided test set.

## Unidirectional RNN Model for document level sentiment classification

In [1]:
import os
import sys
import codecs
import operator
import numpy as np
import re
from time import time
import _pickle as cPickle
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical
import operator
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, Input, Bidirectional
from keras.models import Model
import keras.optimizers as opt
from keras.callbacks import EarlyStopping, ModelCheckpoint

C:\Users\20181592\AppData\Local\Continuum\anaconda3\envs\py3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [2]:
data_path = 'data/doc_level'

In [0]:
num_regex = re.compile('^[+-]?[0-9]+\.?[0-9]*$')

def create_vocab(domain, data_path, maxlen=0, vocab_size=0):
    
    print('Creating vocab ...')

    f = os.path.join(data_path,'%s_text.txt'%(domain))

    total_words, unique_words = 0, 0
    word_freqs = {}

    fin = codecs.open(f, 'r', 'utf-8')
    for line in fin:
        words = line.split()
        if maxlen > 0 and len(words) > maxlen:
            continue

        for w in words:
            if not bool(num_regex.match(w)):
                try:
                    word_freqs[w] += 1
                except KeyError:
                    unique_words += 1
                    word_freqs[w] = 1
                total_words += 1

    print ('  %i total words, %i unique words' % (total_words, unique_words))
    sorted_word_freqs = sorted(word_freqs.items(), key=operator.itemgetter(1), reverse=True)

    vocab = {'<pad>':0, '<unk>':1, '<num>':2}
    index = len(vocab)
    for word, _ in sorted_word_freqs:
        vocab[word] = index
        index += 1
        if vocab_size > 0 and index > vocab_size + 2:
            break
    if vocab_size > 0:
        print (' keep the top %i words' % vocab_size)

  
    return vocab


In [0]:
def create_data(vocab, text_path, label_path, domain, skip_top, skip_len, replace_non_vocab):
    
    data = []
    label = [] # {pos: 0, neg: 1, neu: 2}
    
    f = codecs.open(text_path, 'r', 'utf-8')
    f_l = codecs.open(label_path, 'r', 'utf-8')
    
    num_hit, unk_hit, skip_top_hit, total = 0., 0., 0., 0.
    pos_count, neg_count, neu_count = 0, 0, 0
    max_len = 0

    for line, score in zip(f, f_l):
        word_indices = []
        words = line.split()
        if skip_len > 0 and len(words) > skip_len:
            continue

        score = float(score.strip())
        if score < 3:
            neg_count += 1
            label.append(1)
        elif score > 3:
            pos_count += 1
            label.append(0)
        else:
            neu_count += 1
            label.append(2)
          
        for word in words:
            if bool(num_regex.match(word)):
                word_indices.append(vocab['<num>'])
                num_hit += 1
            elif word in vocab:
                word_ind = vocab[word]
                if skip_top > 0 and word_ind < skip_top + 3:
                    skip_top_hit += 1
                else:
                    word_indices.append(word_ind)
            else:
                if replace_non_vocab:
                    word_indices.append(vocab['<unk>'])
                unk_hit += 1
            total += 1

        if len(word_indices) > max_len:
            max_len = len(word_indices)

        data.append(word_indices)

    f.close()
    f_l.close()

    print('  <num> hit rate: %.2f%%, <unk> hit rate: %.2f%%' % (100*num_hit/total, 100*unk_hit/total))

    print (domain)
    print( 'pos count: ', pos_count )
    print( 'neg count: ', neg_count )
    print( 'neu count: ', neu_count )

    return np.array(data), np.array(label), max_len

In [0]:
def prepare_data(domain, data_path, vocab_size, skip_top=0, skip_len=0, replace_non_vocab=1):
    
    print(domain)

    assert domain in ['amazon_electronics', 'yelp14']

    vocab = create_vocab(domain, data_path, skip_len, vocab_size)
    #print(vocab)

    text_path = os.path.join(data_path,'%s_text.txt'%(domain))
    score_path = os.path.join(data_path,'%s_label.txt'%(domain))

    data, label, max_len = create_data(vocab, text_path, score_path, domain, skip_top, \
                                       skip_len, replace_non_vocab)

    return vocab, data, label, max_len

In [3]:
# choose domain data to train
domain_name = 'amazon_electronics'

In [0]:
vocab, data_list, label_list, overall_maxlen = prepare_data(domain_name, data_path, 10000)

amazon_electronics
Creating vocab ...
  3440972 total words, 39122 unique words
 keep the top 10000 words
  <num> hit rate: 1.04%, <unk> hit rate: 1.56%
amazon_electronics
pos count:  10000
neg count:  10000
neu count:  10000


In [0]:
idx_words = dict((v,k) for (k,v) in vocab.items())

In [4]:
data_path_save = 'Assign3DataStorage/'

In [6]:
def read_pickle(path_data, file_name):

    f = open(os.path.join(path_data, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file

def save_pickle(path_data, file_name, data):

    f = open(os.path.join(path_data, file_name), 'wb')
    cPickle.dump(data, f)
    print(" file saved to: %s"%(os.path.join(path_data, file_name)))
    f.close()

In [0]:
save_pickle(data_path_save, 'words_idx.pkl', vocab)

 file saved to: Assign3DataStorage/words_idx.pkl


In [0]:
save_pickle(data_path_save, 'idx_words.pkl', idx_words)

 file saved to: Assign3DataStorage/idx_words.pkl


In [0]:
save_pickle(data_path_save, 'data.pkl', data_list)

 file saved to: Assign3DataStorage/data.pkl


In [0]:
save_pickle(data_path_save, 'label.pkl', label_list)

 file saved to: Assign3DataStorage/label.pkl


### End of Preprocessing

### Model training, testing and conclusion summary of these are as follows

In [6]:
words_idx = read_pickle(data_path, 'words_idx.pkl')

In [7]:
idx_words = read_pickle(data_path, 'idx_words.pkl')

In [8]:
data = read_pickle(data_path, 'data.pkl')

In [9]:
label = read_pickle(data_path, 'label.pkl')

In [10]:
rand_idx = np.arange(len(data))
np.random.shuffle(rand_idx)

data = data[rand_idx]
label = to_categorical(label)[rand_idx]

data_size = len(data)

test_x = data[0:6000]
test_y = label[0:6000]

dev_x = data[6000:10800]
dev_y = label[6000:10800]

train_x = data[10800:int(data_size)]
train_y = label[10800:int(data_size)]


In [11]:
maxlen = 300
words_idx = [x for (x, _) in sorted(words_idx.items(), key=operator.itemgetter(1))]

In [12]:
train_x_ = sequence.pad_sequences(train_x, maxlen)
dev_x_ = sequence.pad_sequences(dev_x, maxlen)
test_x_ = sequence.pad_sequences(test_x, maxlen)

In [13]:
train_x_ = np.array(train_x_)
train_y = np.array(train_y)

dev_x_ = np.array(dev_x_)
dev_y = np.array(dev_y)

test_x_ = np.array(test_x_)
test_y = np.array(test_y)

In [14]:
class Dataiterator():
    '''
      1) Iteration over minibatches using next(); call reset() between epochs to randomly shuffle the data
      2) Access to the entire dataset using all()
    '''
    
    def __init__(self, X, y, seq_length=32, decoder_dim=300, batch_size=32):      
        self.X = X 
        self.y = y 
        self.num_data = len(X) # total number of examples
        self.batch_size = batch_size # batch size
        self.reset() # initial: shuffling examples and set index to 0
    
    def __iter__(self): # iterates data
        return self


    def reset(self): # initials
        self.idx = 0
        self.order = np.random.permutation(self.num_data) # shuffling examples by providing randomized ids 
        
    def __next__(self): # return model inputs - outputs per batch
        X_ids = [] # hold ids per batch 
        while len(X_ids) < self.batch_size:
            X_id = self.order[self.idx] # copy random id from initial shuffling
            X_ids.append(X_id)
            self.idx += 1 # 
            if self.idx >= self.num_data: # exception if all examples of data have been seen (iterated)
                self.reset()
                raise StopIteration()
        batch_X = self.X[np.array(X_ids)] # X values (encoder input) per batch
        batch_y = self.y[np.array(X_ids)] # y_in values (decoder input) per batch
        return batch_X, batch_y

          
    def all(self): # return all data examples
        return self.X, self.y

## Model

In [17]:
sentence_input = Input(shape=(300,), dtype='int32', name='sentence_input')

In [18]:
vocab_size = len(words_idx)
word_emb = Embedding(vocab_size, 300, mask_zero=True, name='word_emb')
emb_output = word_emb(sentence_input)
drop = Dropout(0.25)(emb_output)

In [0]:
# 1 no embd drop, lstm dropout = 0.5, reccr drop = 0.1
dropout6 = 0.5
recurrent_dropout6 = 0.1
lstm_layer6 = LSTM(300, return_sequences=False, dropout=dropout6, \
              recurrent_dropout=recurrent_dropout6, name='lstm6')(emb_output)

In [0]:
# 2 embd drop = 0.25, lstm dropout = 0.5, reccr drop = 0.1
dropout4 = 0.5
recurrent_dropout4 = 0.1
lstm_layer4 = LSTM(300, return_sequences=False, dropout=dropout4, \
              recurrent_dropout=recurrent_dropout4, name='lstm4')(drop)

In [0]:
# 3 embd drop = 0.25, lstm dropout = 0.5, reccr drop = 0.2
dropout5 = 0.5
recurrent_dropout5 = 0.2
lstm_layer5 = LSTM(300, return_sequences=False, dropout=dropout5, \
              recurrent_dropout=recurrent_dropout5, name='lstm5')(drop)

In [0]:
densed6 = Dense(3, name='dense')(lstm_layer6)
probs6 = Activation('softmax')(densed6)

In [0]:
densed4 = Dense(3, name='dense')(lstm_layer4)
probs4 = Activation('softmax')(densed4)

In [0]:
densed5 = Dense(3, name='dense')(lstm_layer5)
probs5 = Activation('softmax')(densed5)

In [0]:
model6 = Model(inputs=[sentence_input], outputs=probs6)

In [0]:
model4 = Model(inputs=[sentence_input], outputs=probs4)

In [0]:
model5 = Model(inputs=[sentence_input], outputs=probs5)

In [0]:
optimizer = opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)

In [0]:
model6.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print("summary of Model6")
model6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sentence_input (InputLayer)  (None, 300)               0         
_________________________________________________________________
word_emb (Embedding)         (None, 300, 300)          3000900   
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               721200    
_________________________________________________________________
dense (Dense)                (None, 3)                 903       
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 3,723,003
Trainable params: 3,723,003
Non-trainable params: 0
_________________________________________________________________


In [0]:
model4.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print("summary of Model4")
model4.summary()

In [0]:
model5.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print("summary of Model5")
model5.summary()

In [0]:
batch_size = 32

In [0]:
train_x_[1].shape

(300,)

In [0]:
train_steps_epoch = len(train_x_)/batch_size
batch_train_iter = Dataiterator(train_x_, train_y, batch_size)

In [0]:
val_steps_epoch = len(dev_x_)/batch_size
batch_val_iter = Dataiterator(dev_x_, dev_y, batch_size)

In [0]:
test_steps_epoch = len(test_x_)/batch_size
batch_test_iter = Dataiterator(test_x_, test_y, batch_size)

In [0]:
def train_generator(model, batch_train_iter, batch_val_iter):
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=12),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]

    def train_gen():
        while True:
            train_batches = [[X, y] for X, y in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch

    def val_gen():
        while True:
            val_batches = [[X, y] for X, y in batch_val_iter]
            for val_batch in val_batches:
                yield val_batch

    history = model.fit_generator(train_gen(), validation_data=val_gen(), \
                                  validation_steps=val_steps_epoch, steps_per_epoch=train_steps_epoch, \
                                  epochs = 20, callbacks = earlystop_callbacks)


## Training

In [0]:
#Input shape as (300, )
print("Training for model6")
train_generator(model3, batch_train_iter, batch_val_iter)

Epoch 1/20
600/600 [==============================] - 440s 733ms/step - loss: 0.9275 - categorical_accuracy: 0.5528 - val_loss: 0.8156 - val_categorical_accuracy: 0.6204
Epoch 2/20
600/600 [==============================] - 433s 722ms/step - loss: 0.7730 - categorical_accuracy: 0.6562 - val_loss: 0.7854 - val_categorical_accuracy: 0.6360
Epoch 3/20
600/600 [==============================] - 432s 720ms/step - loss: 0.7033 - categorical_accuracy: 0.6928 - val_loss: 0.8685 - val_categorical_accuracy: 0.5979
Epoch 4/20
600/600 [==============================] - 436s 727ms/step - loss: 0.6401 - categorical_accuracy: 0.7302 - val_loss: 0.7833 - val_categorical_accuracy: 0.6458
Epoch 5/20
600/600 [==============================] - 433s 722ms/step - loss: 0.5846 - categorical_accuracy: 0.7596 - val_loss: 0.8080 - val_categorical_accuracy: 0.6475
Epoch 6/20
600/600 [==============================] - 433s 722ms/step - loss: 0.5304 - categorical_accuracy: 0.7837 - val_loss: 0.9257 - val_categoric

In [0]:
print("Training for model4")
train_generator(model4, batch_train_iter, batch_val_iter)

In [0]:
print("Training for model5")
train_generator(model5, batch_train_iter, batch_val_iter)

## Evaluation

In [0]:
#Input shape as (300, )
print("Testing for model6")
loss, accuracy = model3.evaluate(x = test_x_, y = test_y, verbose = 1)

print(loss)
print(accuracy) #was 63.33, re-execute

6000/6000 [==============================] - 42s 7ms/step
1.0987549308141074
0.33866666666666667


In [0]:
print("Testing for model4")
loss, accuracy = model4.evaluate(x = test_x_, y = test_y, verbose = 1)

print(loss)
print(accuracy)

In [0]:
print("Testing for model5")
loss, accuracy = model5.evaluate(x = test_x_, y = test_y, verbose = 1)

print(loss)
print(accuracy)

## Bidirectional RNN Model for document level sentiment classification

## Model

In [15]:
sentence_input = Input(shape = (maxlen, ), dtype = 'int32', name = 'sentence_input')

In [16]:
vocab_size = len(words_idx)
word_emb = Embedding(vocab_size, maxlen, mask_zero=True, name='word_emb')
emb_output = word_emb(sentence_input)
drop = Dropout(0.25)(emb_output)

In [0]:
# 1 no embd drop, lstmdrop = 0.5, recdrop = 0.1
dropout1 = 0.5
recurrent_dropout1 = 0.1
lstm_layer1 = Bidirectional(LSTM(maxlen, return_sequences=False, dropout=dropout1, \
              recurrent_dropout=recurrent_dropout1, name='lstm1'))(emb_output)

In [0]:
# 2 embd drop = 0.25, lstmdrop = 0.5, recdrop = 0.1
dropout2 = 0.5
recurrent_dropout2 = 0.1
lstm_layer2 = Bidirectional(LSTM(maxlen, return_sequences=False, dropout=dropout2, \
              recurrent_dropout=recurrent_dropout2, name='lstm2'))(drop)

In [24]:
# 3 embd drop = 0.25, lstmdrop = 0.5, recdrop = 0.2
dropout3 = 0.5
recurrent_dropout3 = 0.2
lstm_layer3 = Bidirectional(LSTM(maxlen, return_sequences=False, dropout=dropout3, \
              recurrent_dropout=recurrent_dropout3, name='lstm3'))(drop)

In [17]:
# 7 embd drop = 0.25, lstmdrop = 0.5, recdrop = 0.2, merge_mode = 'ave'
dropout7 = 0.5
recurrent_dropout7 = 0.2
lstm_layer7 = Bidirectional(LSTM(maxlen, return_sequences=False, dropout=dropout7, \
              recurrent_dropout=recurrent_dropout7, name='lstm3'), merge_mode = 'ave')(drop)

In [0]:
densed1 = Dense(3, name='dense1')(lstm_layer1)
probs1 = Activation('softmax')(densed1)

In [0]:
densed2 = Dense(3, name='dense2')(lstm_layer2)
probs2 = Activation('softmax')(densed2)

In [0]:
densed3 = Dense(3, name='dense3')(lstm_layer3)
probs3 = Activation('softmax')(densed3)

In [18]:
densed7 = Dense(3, name='dense3')(lstm_layer7)
probs7 = Activation('softmax')(densed7)

In [0]:
model1 = Model(inputs=[sentence_input], outputs=probs1)

In [0]:
model2 = Model(inputs=[sentence_input], outputs=probs2)

In [0]:
model3 = Model(inputs=[sentence_input], outputs=probs3)

In [21]:
model7 = Model(inputs=[sentence_input], outputs=probs7)

In [19]:
optimizer = opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=10, clipvalue=0)

In [0]:
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print("The summary for Model1")
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sentence_input (InputLayer)  (None, 300)               0         
_________________________________________________________________
word_emb (Embedding)         (None, 300, 300)          3000900   
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 600)               1442400   
_________________________________________________________________
dense (Dense)                (None, 3)                 1803      
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 4,445,103
Trainable params: 4,445,103
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print("The summary for Model2")
model2.summary()

In [0]:
model3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print("The summary for Model3")
model3.summary()

In [22]:
model7.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print("The summary for Model7")
model7.summary()

The summary for Model7
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sentence_input (InputLayer)  (None, 300)               0         
_________________________________________________________________
word_emb (Embedding)         (None, 300, 300)          3000900   
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300)               1442400   
_________________________________________________________________
dense3 (Dense)               (None, 3)                 903       
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 4,444,203
Trainable params: 4,444,203
Non-trainable params: 0
___________________________________________

In [23]:
batch_size = 32

In [24]:
train_steps_epoch = len(train_x_)/batch_size
batch_train_iter = Dataiterator(train_x_, train_y, batch_size)

In [25]:
val_steps_epoch = len(dev_x_)/batch_size
batch_val_iter = Dataiterator(dev_x_, dev_y, batch_size)

In [26]:
test_steps_epoch = len(test_x_)/batch_size
batch_test_iter = Dataiterator(test_x_, test_y, batch_size)

In [27]:
def train_generator(model, batch_train_iter, batch_val_iter):
    earlystop_callbacks = [EarlyStopping(monitor='val_loss', patience=10),
                     ModelCheckpoint(filepath=os.path.join('./','{epoch:02d}-{loss:.2f}.check'), \
                                     monitor='val_loss', save_best_only=False, \
                                     save_weights_only=True)
                     ]

    def train_gen():
        while True:
            train_batches = [[X, y] for X, y in batch_train_iter]
            for train_batch in train_batches:
                yield train_batch

    def val_gen():
        while True:
            val_batches = [[X, y] for X, y in batch_val_iter]
            for val_batch in val_batches:
                yield val_batch

    history = model.fit_generator(train_gen(), validation_data=val_gen(), \
                                  validation_steps=val_steps_epoch, steps_per_epoch=train_steps_epoch, \
                                  epochs = 20, callbacks = earlystop_callbacks)


## Training

In [0]:
#Without drop 0.25
print("TRAINING FOR MODEL1")
train_generator(model1, batch_train_iter, batch_val_iter)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
600/600 [==============================] - 1331s 2s/step - loss: 0.8923 - categorical_accuracy: 0.5780 - val_loss: 0.7570 - val_categorical_accuracy: 0.6621
Epoch 2/20
600/600 [==============================] - 1320s 2s/step - loss: 0.7168 - categorical_accuracy: 0.6852 - val_loss: 0.7202 - val_categorical_accuracy: 0.6737
Epoch 3/20
600/600 [==============================] - 1320s 2s/step - loss: 0.6425 - categorical_accuracy: 0.7255 - val_loss: 0.7411 - val_categorical_accuracy: 0.6615
Epoch 4/20
600/600 [==============================] - 1322s 2s/step - loss: 0.5686 - categorical_accuracy: 0.7628 - val_loss: 0.7206 - val_categorical_accuracy: 0.6817
Epoch 5/20
600/600 [==============================] - 1352s 2s/step - loss: 0.5077 - categorical_accuracy: 0.7946 - val_loss: 0.8369 - val_categorical_accuracy: 0.6725
Epoch 6/20
600/600 [==============================] - 1356s 2s/step - loss: 0.4487 - categorical_accuracy: 0.823

In [0]:
print("TRAINING FOR MODEL2")
train_generator(model2, batch_train_iter, batch_val_iter)

In [0]:
print("TRAINING FOR MODEL3")
train_generator(model3, batch_train_iter, batch_val_iter)

In [28]:
print("TRAINING FOR MODEL7")
train_generator(model7, batch_train_iter, batch_val_iter)

TRAINING FOR MODEL3
Epoch 1/20
600/600 [==============================] - 783s 1s/step - loss: 0.9205 - categorical_accuracy: 0.5572 - val_loss: 0.8036 - val_categorical_accuracy: 0.6181
Epoch 2/20
600/600 [==============================] - 783s 1s/step - loss: 0.7501 - categorical_accuracy: 0.6650 - val_loss: 0.7679 - val_categorical_accuracy: 0.6400
Epoch 3/20
600/600 [==============================] - 797s 1s/step - loss: 0.6825 - categorical_accuracy: 0.7056 - val_loss: 0.7427 - val_categorical_accuracy: 0.6698
Epoch 4/20
600/600 [==============================] - 786s 1s/step - loss: 0.6252 - categorical_accuracy: 0.7358 - val_loss: 0.7199 - val_categorical_accuracy: 0.6687
Epoch 5/20
600/600 [==============================] - 786s 1s/step - loss: 0.5811 - categorical_accuracy: 0.7544 - val_loss: 0.7424 - val_categorical_accuracy: 0.6660
Epoch 6/20
600/600 [==============================] - 961s 2s/step - loss: 0.5289 - categorical_accuracy: 0.7839 - val_loss: 0.7496 - val_categor

### Evaluate

In [0]:
#Without drop 0.25
print("TESTING FOR MODEL1")
loss, accuracy = model1.evaluate(x = test_x_, y = test_y, verbose = 1)

print(loss)
print(accuracy) # test acc is 64.9%

6000/6000 [==============================] - 90s 15ms/step
1.3527190974553427
0.649


In [0]:
print("TESTING FOR MODEL2")
loss, accuracy = model2.evaluate(x = test_x_, y = test_y, verbose = 1)

print(loss)
print(accuracy)

In [0]:
print("TESTING FOR MODEL3")
loss, accuracy = model3.evaluate(x = test_x_, y = test_y, verbose = 1)

print(loss)
print(accuracy)

In [29]:
print("TESTING FOR MODEL7")
loss, accuracy = model7.evaluate(x = test_x_, y = test_y, verbose = 1)

print(loss)
print(accuracy)

TESTING FOR MODEL7
6000/6000 [==============================] - 67s 11ms/step
1.1489095962842306
0.6581666666666667


### Summary of the model


#### UniDirectional RNN

model6: acc:61.45 (no embd drop, lstm dropout = 0.5, reccr drop = 0.1), approx. training time: 96mins

model4: acc:64.03 (embd drop = 0.25, lstm dropout = 0.5, reccr drop = 0.1), approx. training time: 75mins

model5: acc:61.75 (embd drop = 0.25, lstm dropout = 0.5, reccr drop = 0.2), approx. training time: 85mins


#### BiDirectional RNN

model1: acc:61.38 (no embd drop, lstmdrop = 0.5, recdrop = 0.1), approx. training time: 145mins

model2: acc:63.93 (embd drop = 0.25, lstmdrop = 0.5, recdrop = 0.1), approx. training time: 145mins

model3: acc:63.88 (embd drop = 0.25, lstmdrop = 0.5, recdrop = 0.2), approx. training time: 140mins

model7: acc:65.81 (embd drop = 0.25, lstmdrop = 0.5, recdrop = 0.2, merge_mode = 'ave'), approx. training time: 150mins

We can see that in Uni Directional when having dropout for each embedding layer, lastm layer and reccurent dropout we get the highest accuracy during evaluation as overfitting in the training set has been reduced and thus the model is able to predict the unseen with higher accuracy. Also has less training time than the other models as the number of neurons that are less significant are dropped out.

In Bi Directional we can see that model7 achieves high accuracy with merge mode as average but takes more time for training. Considering the tradeoff between accuracy and training time we can conclude that the configuration in model2 is best performing.

The above reported are the models that give some meaningful change/insight into the performance of the model for small variation in hyper parameters among all the other models that were tried.